In [1]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4
from scrape_aircraft_list import scrape_aircraft_list
from get_perf_model_typecodes import get_perf_model_typecodes 
from match_icao_model import match_icao_model
from process_airport_list import process_airport_list
from generate_flightpath import generate_flightpath
from plot_flightpaths import plot_flightpaths
from get_engine_data import get_engine_data
from perf_model_powerplant_parser import perf_model_powerplant_parser
import pandas as pd
import numpy as np
from time import sleep
from geopy import distance
from icet import icet



In [2]:
# User Inputs:
start_time_str       = '2024-01-01T00:00:00Z'
stop_time_str        = '2024-12-31T23:59:59Z'
query_limit          = 30e4
send_notification    = True
make_plot            = False
output_dir           = "/scratch/omg28/Data/no_track_2024"

# Convert start and stop times to datetime objects
start_time_simple = pd.to_datetime(start_time_str).strftime("%Y-%m-%d")
stop_time_simple = pd.to_datetime(stop_time_str).strftime("%Y-%m-%d")

Section 1. Download flight information from Opensky

In [ ]:
"""
# Download data from OpenSky history database
# WARNING: This may take a long time to run and will use a lot of disk space. Recommend running this in a .py script to avoid recurring memory issues found in Jupyter notebooks.
download_day(start_time_str, stop_time_str, query_limit, send_notification, make_plot, output_dir)
"""

In [ ]:
# load the saved day of data from a .pkl file to a pandas dataframe
loaded_day = load_saved_fd4(stop_time_str, output_dir, query_limit)

Section 2. Scrape the EUCONTROL database for aircraft flight performance information

In [ ]:
"""
# scrape the list of aircraft with performance models from the EUCONTROL website
perf_model_typecodes = get_perf_model_typecodes()
## WARNING: THIS CAN GET YOUR IP ADDRESS BLOCKED IF YOU RUN IT REPEATEDLY. IT IS A WEB SCRAPER.
aircraft_list = scrape_aircraft_list(perf_model_typecodes)

"""

In [ ]:
# remove the helicopters from the aircraft performance database list
perf_models = pd.read_csv("aircraft_performance_table.csv")
perf_models = perf_models[~perf_models['type'].str.contains('H')]
perf_models.to_csv("aircraft_performance_table_filtered.csv", index=False)
pd.to_pickle(perf_models, "aircraft_performance_table_filtered.pkl")


Section 3. Process the aircraft performance and flight data, retaining only flights we have takeoff, landing, plane type, and aircraft performance model information about. Further, drop all flights with aircraft codes that do not correspond to a code in the ourairports registry.

In [ ]:
# process the loaded time period of data. - 20s
# - matches 24 bit transponder codes from Feb 2025 aircraft metadata database with the ICAO typecode
# - removes all flights for which the transponder code does not have a corresponding ICAO typecode
# - removes all flights for which the takeoff OR landing airport is unknown
flights_with_typecode = match_icao_model(start_time_str, stop_time_str, query_limit)


In [ ]:
# Load in the list of ICAO typecodes that have a performance model
flights_with_typecode = pd.read_pickle('/scratch/omg28/Data/aircraftdb/2024-01-01_to_2024-12-31_300000.pkl')
# load in the list of all flights with typecode aand takeoff/landing airport from 2024
performance_model_typecodes = pd.read_pickle('/scratch/omg28/Data/aircraftdb/performance_models_typecodes.pkl')


# get info before the merge
print("Before merging:")
print(flights_with_typecode.info())

# inner join the loaded year of data with the scraped aircraft list
# - this will remove all flights for which the ICAO typecode does not have a corresponding performance model
flights_with_perf_typecode = pd.merge(flights_with_typecode, performance_model_typecodes, how='inner', on = 'typecode')

print("After merging:")
print(flights_with_perf_typecode.info())


In [ ]:
# print a list of typecodes in complete_flights_perf_model that do not have a performance model
missing_perf_models = flights_with_typecode[~flights_with_typecode['typecode'].isin(performance_model_typecodes['typecode'])]
print("Aircraft models with observed flights, but no performance model: " + str(len(missing_perf_models['typecode'].unique())))
print("total flights in 2024 with TOLD and typecode, but no performance model: " + str(len(missing_perf_models)))


In [ ]:

# print a list of typecodes in performance_model_typecodes that do not correspond to a flight
missing_flights = performance_model_typecodes[~performance_model_typecodes['typecode'].isin(flights_with_typecode['typecode'])]
print("Number of performance models with no flights: " + str(len(missing_flights['typecode'].unique())))
print("Number of flights with TOLD, typecode, and performance model: " + str(len(flights_with_perf_typecode)))


In [ ]:
# count all flights starting and ending at the same airport
num_flights_no_dist = len(flights_with_perf_typecode[flights_with_perf_typecode['estdepartureairport'] == flights_with_perf_typecode['estarrivalairport']])
print("Number of flights with no distance: " + str(num_flights_no_dist))

# remove all flights starting or ending at the same airport
flights_with_perf_typecode_dist = flights_with_perf_typecode[flights_with_perf_typecode['estdepartureairport'] != flights_with_perf_typecode['estarrivalairport']]
print("Number of flights with TOLD, typecode, performance model, and nonzero distance: " + str(len(flights_with_perf_typecode)))


In [ ]:
# load the takeoff and landing airport location information to the dataframe.
all_airports = process_airport_list()

# add the estimated departure airport location information to the dataframe
# - this will remove all flights for which the departure airport is unknown
flights_with_perf_typecode_dist_dep = pd.merge(flights_with_perf_typecode_dist, all_airports, how='inner', left_on = 'estdepartureairport', right_on = 'ident')
flights_with_perf_typecode_dist_dep.rename(columns = {'latitude_deg': 'estdeparturelat', 'longitude_deg': 'estdeparturelong', 'elevation_ft': 'estdeparturealt_ft'}, inplace=True)
flights_with_perf_typecode_dist_dep.drop(columns = ['ident', 'gps_code'], inplace=True)
# count the number of flights with no departure airport in the database
print("Number of flights with no departure airport location info in database: " + str(len(flights_with_perf_typecode_dist) - len(flights_with_perf_typecode_dist_dep)))

# add the estimated arrival airport location information to the dataframe
all_flights_filtered = pd.merge(flights_with_perf_typecode_dist_dep, all_airports, how='inner', left_on = 'estarrivalairport', right_on = 'ident')
all_flights_filtered.rename(columns = {'latitude_deg': 'estarrivallat', 'longitude_deg': 'estarrivallong', 'elevation_ft': 'estarrivalalt_ft'}, inplace=True)
all_flights_filtered.drop(columns = ['ident', 'gps_code', 'id_x', 'id_y'], inplace=True)
# count the number of flights with no arrival airport in the database
print("Number of flights with no arrival airport location info in database: " + str(len(flights_with_perf_typecode_dist_dep) - len(all_flights_filtered)))
print("Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info: " + str(len(all_flights_filtered)))


Section 4. Using the great circle distance, calculate the distance between the airport takeoff and landing.

In [ ]:
# calculate the great circle distance between the departure and arrival airports - 4 min
all_flights_filtered['gc_km'] = all_flights_filtered.apply(lambda x: distance.great_circle((x['estdeparturelat'], x['estdeparturelong']), (x['estarrivallat'], x['estarrivallong'])).km, axis=1)
# adjust great circle distance using eq. 6 of seymour, et al (2020)
# all_flights_filtered = pd.read_pickle(f'/scratch/omg28/Data/no_track_2024/{start_time_simple}_to_{stop_time_simple}_filtered.pkl')

In [ ]:
# could be sped up by using a vectorized function # 1 min
all_flights_filtered['gc_FEAT_km'] = all_flights_filtered.apply(lambda x: 1.0387 * x['gc_km'] + 40.5, axis = 1)

In [ ]:
# save the merged dataframe to a .pkl file
all_flights_filtered.to_pickle(f'/scratch/omg28/Data/no_track2024/{start_time_simple}_to_{stop_time_simple}_filtered.pkl')

In [ ]:
print(all_flights_filtered.info())
print(all_flights_filtered.head())

In [ ]:
# remove all flights under 200 km:
all_flights_filtered = all_flights_filtered[all_flights_filtered['gc_FEAT_km'] > 200]
print("Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info over 200 km: " + str(len(all_flights_filtered)))

Section 5: generate and visualize flightpaths

In [ ]:
performance_data=pd.read_pickle('aircraft_performance_data_table.pkl')
flightpath = generate_flightpath('A320', 500, performance_data)

In [ ]:
import matplotlib.pyplot as plt

# Sum durations from the flightpath dictionary for each phase
climb_time = (flightpath['climb'].get('t_climb_0_5', 0) +
              flightpath['climb'].get('t_climb_5_10', 0) +
              flightpath['climb'].get('t_climb_10_15', 0) +
              flightpath['climb'].get('t_climb_15_24', 0) +
              flightpath['climb'].get('t_climb_ceil', 0))

descent_time = (flightpath['descent'].get('t_descent_5_0', 0) +
                flightpath['descent'].get('t_descent_10_5', 0) +
                flightpath['descent'].get('t_descent_15_10', 0) +
                flightpath['descent'].get('t_descent_24_15', 0) +
                flightpath['descent'].get('t_descent_ceil', 0))

cruise_time = flightpath['cruise'].get('t_cruise', 0)

# Prepare data for the barchart
phases = ['Climb', 'Cruise', 'Descent']
durations = [climb_time, cruise_time, descent_time]

# Optionally convert seconds to minutes (or leave in seconds)
# durations = [t / 60.0 for t in durations]

plt.figure()
bars = plt.bar(phases, durations, color=['skyblue', 'lightgreen', 'salmon'])
plt.xlabel('Flight Phases')
plt.ylabel('Duration (seconds)')
plt.title('Duration of Each Flight Leg')

# Annotate bars with duration values
for bar, duration in zip(bars, durations):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height,
             f'{duration:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()
# Break down the climb phases
climb_labels = ['Climb 0-5', 'Climb 5-10', 'Climb 10-15', 'Climb 15-24', 'Climb Ceil']
climb_times = [
    flightpath['climb'].get('t_climb_0_5', 0),
    flightpath['climb'].get('t_climb_5_10', 0),
    flightpath['climb'].get('t_climb_10_15', 0),
    flightpath['climb'].get('t_climb_15_24', 0),
    flightpath['climb'].get('t_climb_ceil', 0)
]

# Break down the descent phases
descent_labels = ['Descent 5-0', 'Descent 10-5', 'Descent 15-10', 'Descent 24-15', 'Descent Ceil']
descent_times = [
    flightpath['descent'].get('t_descent_5_0', 0),
    flightpath['descent'].get('t_descent_10_5', 0),
    flightpath['descent'].get('t_descent_15_10', 0),
    flightpath['descent'].get('t_descent_24_15', 0),
    flightpath['descent'].get('t_descent_ceil', 0)
]

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))

# Climb segments chart
plt.subplot(1, 2, 1)
bars_climb = plt.bar(climb_labels, climb_times, color='skyblue')
plt.xlabel('Climb Segments')
plt.ylabel('Time (seconds)')
plt.title('Climb Phase Breakdown')
for bar, t in zip(bars_climb, climb_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{t:.1f}', ha='center', va='bottom')

# Descent segments chart
plt.subplot(1, 2, 2)
bars_descent = plt.bar(descent_labels, descent_times, color='salmon')
plt.xlabel('Descent Segments')
plt.ylabel('Time (seconds)')
plt.title('Descent Phase Breakdown')
for bar, t in zip(bars_descent, descent_times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{t:.1f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Overall phase distances (in meters)
climb_distance = (np.atleast_1d(flightpath['climb'].get('s_climb_0_5', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_5_10', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_10_15', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_15_24', 0))[0] +
                  np.atleast_1d(flightpath['climb'].get('s_climb_ceil', 0))[0])

cruise_distance = np.atleast_1d(flightpath['cruise'].get('s_cruise', 0))[0]

descent_distance = (np.atleast_1d(flightpath['descent'].get('s_descent_5_0', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_10_5', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_15_10', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_24_15', 0))[0] +
                    np.atleast_1d(flightpath['descent'].get('s_descent_ceil', 0))[0])

phases = ['Climb', 'Cruise', 'Descent']
distances = [climb_distance, cruise_distance, descent_distance]

plt.figure(figsize=(8, 6))
overall_bars = plt.bar(phases, distances, color=['skyblue', 'lightgreen', 'salmon'])
plt.xlabel('Flight Phases')
plt.ylabel('Distance (meters)')
plt.title('Distance Covered in Each Flight Phase')

# Annotate overall bars with distance values
for bar, dist in zip(overall_bars, distances):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height, f'{dist:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


# Breakdown of distances for each segment

# For Climb Phase
climb_keys = ['s_climb_0_5', 's_climb_5_10', 's_climb_10_15', 's_climb_15_24', 's_climb_ceil']
climb_distances = []
for key in climb_keys:
    value = flightpath['climb'].get(key, np.array([0]))
    # value is expected to be a numpy array; extract its first element if so
    if isinstance(value, np.ndarray):
        climb_distances.append(value[0])
    else:
        climb_distances.append(value)

# For Descent Phase
descent_keys = ['s_descent_5_0', 's_descent_10_5', 's_descent_15_10', 's_descent_24_15', 's_descent_ceil']
descent_distances = []
for key in descent_keys:
    value = flightpath['descent'].get(key, np.array([0]))
    if isinstance(value, np.ndarray):
        descent_distances.append(value[0])
    else:
        descent_distances.append(value)

plt.figure(figsize=(14, 6))

# Climb segments chart
plt.subplot(1, 2, 1)
bars_climb = plt.bar(climb_labels, climb_distances, color='skyblue')
plt.xlabel('Climb Segments')
plt.ylabel('Distance (meters)')
plt.title('Climb Phase Distance Breakdown')
for bar, d in zip(bars_climb, climb_distances):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{d:,.0f}', ha='center', va='bottom')

# Descent segments chart
plt.subplot(1, 2, 2)
bars_descent = plt.bar(descent_labels, descent_distances, color='salmon')
plt.xlabel('Descent Segments')
plt.ylabel('Distance (meters)')
plt.title('Descent Phase Distance Breakdown')
for bar, d in zip(bars_descent, descent_distances):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{d:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()


In [ ]:
# Build cumulative distance and altitude lists.
# Starting at runway: (distance, altitude) = (0, 0)
distances_list = [0]
altitudes_list = [0]

# Climb phase:
# First climb segment (0-5):
cum_distance = distances_list[-1]
for key in flightpath:
    for key2 in flightpath[key]:
        if key2.startswith('s_'):
            cum_distance += flightpath[key][key2]
            distances_list.append(cum_distance)
            altitudes_list.append(flightpath[key][f'{key2.replace('s_', 'h_')}_end'])

# Plot the altitude profile vs. cumulative flight distance.
plt.figure(figsize=(10, 6))
plt.plot(distances_list, altitudes_list, marker='o', linestyle='-')
plt.xlabel('Cumulative Flight Distance (meters)')
plt.ylabel('Flight Altitude (meters)')
plt.title('Flight Altitude vs. Flight Distance')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate 25 flight distances (km) linearly spaced from 200 to 5000 km.
num_flightpaths = 25
flight_distances = np.linspace(200, 1000, num_flightpaths)
flightpaths = [0] * num_flightpaths
 
# Loop through each flight distance and generate the flightpath.
for ii in np.arange(num_flightpaths):
    d_km = flight_distances[ii] 
    # Generate flightpath for a given flight distance (in km)
    fp = generate_flightpath('C130', d_km, performance_data)
    flightpaths[ii] = fp
plot_flightpaths(flightpaths)

Section 6: parse the engine data and emissions data and add it to the aircraft performance model.

In [2]:
engine_models = get_engine_data()

Data loaded from /scratch/omg28/Data/engine_data/engine_data_icao.csv and saved to /scratch/omg28/Data/engine_data/engine_data_icao.pkl


In [3]:
performance_models = pd.read_csv("aircraft_performance_table_filtered.csv")

In [ ]:
performance_models = perf_model_powerplant_parser(performance_models)
performance_models_debug = performance_model[['powerplant', 'number_of_engines', 'thrust', 'engine code', 'manufacturer']].copy()

None
